### LLM Model Demo

In [ ]:
# Installed LangChain 
#%pip install langchain langchain-community faiss-cpu langchain-chroma langchain-openai langchain_text_splitters langgraph beautifulsoup4 langchain-huggingface

In [ ]:
#%pip install langchain_huggingface

In [8]:
import langchain
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader # type: ignore
from langchain.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain.chains import RetrievalQA # type: ignore
from langchain_ollama.llms import OllamaLLM # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
from langchain_core.messages import HumanMessage, SystemMessage # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore

from langchain.cache import InMemoryCache

In [9]:
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"


In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version = os.environ["OPENAI_API_VERSION"],
    temperature= 0.3
)
langchain.llm_cache = InMemoryCache() # type: ignore
parser = StrOutputParser()
#embeddings = HuggingFaceEmbeddings() #model="sentence-transformers/all-mpnet-base-v2"

In [ ]:
messages = [
    SystemMessage(content="Translate the following from English into Bengali"),
    HumanMessage(content="hi!"),
]

In [11]:
prompt = "What is current date and time in India?"

In [6]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [12]:
result = llm.invoke(prompt)
parser.invoke(result)

"I don't have real-time capabilities to provide the current date and time. However, you can easily find the current date and time in India by checking a reliable world clock website, using a smartphone, or asking a digital assistant like Siri or Google Assistant. India is in the Indian Standard Time (IST) zone, which is UTC+5:30."

In [8]:
result = llm.invoke(messages)
parser.invoke(result)

'হ্যালো!'

In [ ]:
chain = prompt_template | llm | parser
chain.invoke({"language": "bengali", "text": "how are you doing"})

In [ ]:
#%pip install langchain_ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(
    model="llama3.2",
    top_k=3
    )

chain = prompt | model

chain.invoke({"question": "How far Earth from Sun?"})

"To calculate the average distance of Earth from the Sun, we'll consider a few key factors:\n\n1. **Astronomical Unit (AU)**: The AU is a standard unit used to measure distances in our solar system. It's equal to the average distance between the Earth and the Sun.\n\n2. **Orbit of the Earth**: The Earth orbits the Sun at an average speed of about 29.78 kilometers per second (km/s). This speed, combined with its orbital period of approximately 365.25 days, allows us to calculate its average distance from the Sun.\n\nNow, let's use these values to find the answer:\n\nAverage distance (in AU) = Average speed / Orbital period\n\nFirst, we need to convert the speed from km/s to AU per day. There are 24 hours in a day and approximately 365.25 days in a year.\n\nSpeed in AU/day ≈ 29.78 km/s × (1 AU / 149,597,890 km) × (1 day / 24 hours) ≈ 0.0000167 AU/day\n\nNow we can calculate the average distance:\n\nAverage distance = Average speed / Orbital period\n= 0.0000167 AU/day ÷ 365.25 days\n≈ 0.0

In [ ]:
%pip install langchain faiss-cpu unstructured unstructured[pdf]

In [ ]:
# filepath: rag_implementation.py


# Load documents from a local folder
def load_documents_from_folder(folder_path):
    #loader = DirectoryLoader(folder_path,glob="**/*.pdf",show_progress=True)
    loader = PyPDFDirectoryLoader(folder_path)
    documents = loader.load()
    return documents

# Embed and vectorize documents using FAISS
def embed_and_vectorize_documents(documents):
    embeddings = OllamaEmbeddings(model="llama3.2")
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# Implement RAG
def implement_rag(vector_store, query):
    llm  = OllamaLLM(
    model="llama3.2"
    )
    retriever = vector_store.as_retriever()
    rag_chain = RetrievalQA(llm=llm, retriever=retriever)
    result = rag_chain.run(query)
    return result

messages = [
    SystemMessage(content="As Technical Recruiter Evaluate the following resumes"),
    HumanMessage(content="Hello Recruiter"),
]

system_template = "Translate the following into {language}:"

"""prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
) """

if __name__ == "__main__":
    folder_path = r"C:\Users\HSASS\OneDrive - Wipro\Desktop\Temp2\Resume"
    query = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")])

    # Load documents
    documents = load_documents_from_folder(folder_path)

    # Embed and vectorize documents
    vector_store = embed_and_vectorize_documents(documents)

    # Implement RAG and get the result
    result = implement_rag(vector_store, query)
    print(result)
    